In [ ]:
#import the libraries:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
import cv2
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from random import shuffle
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold

In [ ]:
# unzipping the file:
!unzip /content/BAU_HW_CAT_DOG_CLASSIFICATION.zip

In [ ]:
# implementing the preprocessing functions: 

import os
import cv2
import numpy as np

def load_images_labels(dir, label):
    """Loads images and labels from a directory.

    Args:
        dir: string, path to the directory to load images from.
        label: int, label for the images.

    Returns:
        images: list of NumPy arrays, images in the directory.
        labels: list of ints, labels for the images.
    """
    images = []
    labels = []

        image = cv2.imread(os.path.join(dir, file))
        image = cv2.resize(image, (200, 200))
        images.append(image)
        labels.append(label)

    return images, labels

def normalize_images(images):
    """Normalizes images.

    Args:
        images: list of NumPy arrays, images to normalize.

    Returns:
        normalized_images: list of NumPy arrays, normalized images.
    """
    normalized_images = []
    for image in images:
        normalized_images.append(image / 255.0)
    return normalized_images

def combine_images_labels(images_1, labels_1, images_2, labels_2):
    """Concatenates images and labels.

    Args:
        images_1: list of NumPy arrays, first set of images.
        labels_1: list of ints, first set of labels.
        images_2: list of NumPy arrays, second set of images.
        labels_2: list of ints, second set of labels.

    Returns:
        combined_images: NumPy array, concatenation of images_1 and images_2.
        combined_labels: NumPy array, concatenation of labels_1 and labels_2.
    """
    combined_images = np.concatenate((images_1, images_2))
    combined_labels = np.concatenate((labels_1, labels_2))
    return combined_images, combined_labels

# implementing the data paths:
train_dir = '/content/BAU_HW_CAT_DOG_CLASSIFICATION/training_set'
test_dir = '/content/BAU_HW_CAT_DOG_CLASSIFICATION/test_set'

# loading and normalizing images and labels for the training set:


x_train_cats, y_train_cats = load_images_labels(
os.path.join(train_dir, 'cats'), 1)
x_train_cats = normalize_images(x_train_cats)

x_train_dogs, y_train_dogs = load_images_labels(
os.path.join(train_dir, 'dogs'), 0)
x_train_dogs = normalize_images(x_train_dogs)

# normalization 

x_test_cats, y_test_cats = load_images_labels(
os.path.join(test_dir, 'cats'), 1)
x_test_cats = normalize_images(x_test_cats)

x_test_dogs, y_test_dogs = load_images_labels(
os.path.join(test_dir, 'dogs'), 0)
x_test_dogs = normalize_images(x_test_dogs)

# concatenatination

x_train, y_train = combine_images_labels(
x_train_cats, y_train_cats, x_train_dogs, y_train_dogs)

#making training and testing sets:

x_test, y_test = combine_images_labels(
x_test_cats, y_test_cats, x_test_dogs, y_test_dogs)
x_valid, y_valid = x_train[:1000], y_train[:1000]
x_train, y_train = x_train[1000:], y_train[1000:]



In [ ]:
# Model 1:
model = models.Sequential()

# Convolution Layers:
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

#Fully connected layers and output:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

# Compiling:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Kfold with 5folds
kfold = KFold(n_splits=5, shuffle=True)

# Istoring the predictions:
predictions = []



In [ ]:

for train_index, test_index in kfold.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    history = model.fit(x_train_fold, y_train_fold, epochs= 20, batch_size=64, validation_data=(x_test_fold, y_test_fold))
    test_loss, test_acc = model.evaluate(x_test_fold, y_test_fold)
    print('Fold test accuracy:', test_acc)

    # Plotting as required: 

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.show()

    # Evaluation on test data
    _, accuracy = model.evaluate(x_test_fold, y_test_fold)
    predictions.append(accuracy)
    



In [ ]:
#Evaluation on test data:

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

#Evaluation on train data:
train_loss, train_acc = model.evaluate(x_train, y_train)
print('Train accuracy:', train_acc)

# mean accuracy across of the 5 folds
mean_accuracy = np.mean(predictions)
print('Mean accuracy:', mean_accuracy)

In [ ]:
# Model 2: Since the models and evaluation are the construvtion, no need for the same comments again:

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='sigmoid')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

kfold = KFold(n_splits=5, shuffle=True)
predictions = []


In [ ]:

for train_index, test_index in kfold.split(x_train):
    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    history = model.fit(x_train_fold, y_train_fold, epochs=20, batch_size=64, validation_data=(x_test_fold, y_test_fold))
    test_loss, test_acc = model.evaluate(x_test_fold, y_test_fold)
    print('Fold test accuracy:', test_acc)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.show()

    _, accuracy = model.evaluate(x_test_fold, y_test_fold)
    predictions.append(accuracy)


In [ ]:

mean_accuracy = np.mean(predictions)
print('Mean accuracy:', mean_accuracy)

train_loss, train_acc = model.evaluate(x_train, y_train)
print('Training accuracy:', train_acc)

test_loss, test_acc = model.evaluate(x_test, y_test)
print('Testing accuracy:', test_acc)

In [ ]:
# Model 3: 

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='sigmoid'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='sigmoid')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)
predictions = []




In [ ]:

for train_index, test_index in kfold.split(x_train):

    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
  
    history = model.fit(x_train_fold, y_train_fold, epochs=20, batch_size=64, validation_data=(x_test_fold, y_test_fold))
    test_loss, test_acc = model.evaluate(x_test_fold, y_test_fold)
    print('Fold test accuracy:', test_acc)

    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.show()

    
    _, accuracy = model.evaluate(x_test_fold, y_test_fold)

    
    predictions.append(accuracy)



In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Testing accuracy:', test_acc)


train_loss, train_acc = model.evaluate(x_train, y_train)
print('Training accuracy:', train_acc)


mean_accuracy = np.mean(predictions)
print('Mean accuracy:', mean_accuracy)


In [ ]:
# Model 4: 

resnet = ResNet50(include_top=False, input_shape=(200, 200, 3))
model = Sequential()
model.add(resnet)
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) 

kfold = KFold(n_splits=5, shuffle=True)
predictions = []




In [ ]:

for train_index, test_index in kfold.split(x_train):

    x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    
    history = model.fit(x_train_fold, y_train_fold, epochs=20, batch_size=64, validation_data=(x_test_fold, y_test_fold))


    test_loss, test_acc = model.evaluate(x_test_fold, y_test_fold)
    print('Fold test accuracy:', test_acc)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Training Loss', 'Validation Loss'])
    plt.show()
    _, accuracy = model.evaluate(x_test_fold, y_test_fold)
    predictions.append(accuracy)


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Testing accuracy:', test_acc)


train_loss, train_acc = model.evaluate(x_train, y_train)
print('Training accuracy:', train_acc)


mean_accuracy = np.mean(predictions)
print('Mean accuracy:', mean_accuracy)